In [3]:
# train a generative adversarial network on a one-dimensional function
from numpy import hstack
from numpy import zeros
from numpy import ones
from numpy.random import rand
from numpy.random import randn
from keras.models import Sequential
from keras.layers import Dense
from matplotlib import pyplot
import requests
import numpy as np
import pandas as pd
import json

from bs4 import BeautifulSoup


In [4]:
#file1 = open('cards-standard.txt', 'r') 
#Lines = file1.readlines() 
#maximum = len(Lines)
#cards_key = {}
#cards_value = {}
#i=1
#for line in Lines:
 #   cards_value[i] = line.rstrip("\n")
  #  cards_key[line.rstrip("\n")] = i
   # i += 1

#print(cards_key["Arcane Breath"])
#print(cards_value[32])
#print(len(Lines))

In [5]:
classes_id = {'Demon Hunter' : 16384,
          'Druid' : 4,
          'Hunter' : 8,
          'Mage' : 16,
          'Paladin' : 32,
          'Priest' : 64, 
          'Rogue' : 128,
          'Shaman' : 256,
          'Warlock' : 512,
          'Warrior' : 1024}

r = requests.get('https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&filter-class='+str(classes_id['Demon Hunter']))
soup = BeautifulSoup(r.content,"html.parser")
rows = soup.find('ul', class_="b-pagination-list paging-list j-tablesorter-pager j-listing-pagination")
links = rows.find_all('li')
print(links[len(links)-2].text)


58


In [6]:
#scrape deck data from Hearthpwn website, standard decks

def page_max(deck_type):
    r = requests.get('https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&filter-class='+str(classes_id[deck_type]))
    soup = BeautifulSoup(r.content,"html.parser")
    rows = soup.find('ul', class_="b-pagination-list paging-list j-tablesorter-pager j-listing-pagination")
    links = rows.find_all('li')
    return int(links[len(links)-2].text)

def scrape_decks(deck_type):
    start_urls = [] 
    n = page_max(deck_type)
    print('Number of Pages: ',n)
    for i in range(n):
        start_urls.append("https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=" + str(i+1)+"&filter-class="+ str(classes_id[deck_type]))
    #print(start_urls)
    deck_urls = []
    deck_classes = []
    for url in start_urls:
        print(url)
        r = requests.get(url)
        soup = BeautifulSoup(r.content,"html.parser")
        #rows = soup.find(id="decks-container")
 
        rows = soup.find_all('table', class_="listing listing-decks b-table b-table-a")
        for row in rows:
            decks = row.find_all('span', class_="tip")
            for deck in decks:
                links = deck.find_all('a', href=True)
                for link in links:
                    deck_urls.append("https://www.hearthpwn.com" + link['href'])
                    
            deck_class = row.find_all('td', class_="col-class")
            for cl in deck_class:
                deck_classes.append(cl.text)

    return cards_from_url(deck_urls), deck_classes



In [7]:
#scrape cards in deck from separate url
def cards_from_url(deck_urls):
    decks = []
    for url in deck_urls:
        print(url)
        deck=[]
        r = requests.get(url)
        soup = BeautifulSoup(r.content,"html.parser")
        table = soup.find_all('table', class_="listing listing-cards-tabular b-table b-table-a")
        for rows in table:
            row = rows.find_all('td', class_="col-name")
            for i in row:
                card = i.find('a')
                name = card.text
                number = card['data-count']

                for num in range(int(number)):
                    deck.append(name.replace('\n',''))
        #print(deck)
        #print(len(deck))
        decks.append(deck)
    return decks


        


In [8]:
def deck_to_vector(deck):
    deck_vector = []
    for elem in deck:
        #remove any potential white spaces from strings
        #if type(deck) is not list:
        elem = elem.lstrip()
        elem = elem.rstrip()

        deck_vector.append(cards_key[elem])
    return deck_vector

def vector_to_deck(deck_vector):
    deck = []
    for elem in deck_vector:
        deck.append(cards_value[elem])
    return deck

def vectorize_deck_list(decks):
    vecks = []
    for d in decks:
        vecks.append(deck_to_vector(d))
    return vecks

def concatenate_deck_class(decks, classes):
    concatenated_result = []
    for i in range(len(decks)):
        concatenated_result.append([decks[i],classes[i]])
    return concatenated_result



In [9]:
#deck_type = ['Demon Hunter','Druid','Hunter','Mage','Paladin','Priest','Rogue','Shaman','Warlock','Warrior']
#deck_type = ['Hunter','Mage','Paladin','Priest','Rogue','Shaman','Warlock']
#deck_type = ['Paladin','Priest','Rogue','Shaman','Warlock']

#for cl in deck_type:
 #   print("Scraping information: ", cl)
  #  decks, deck_classes = scrape_decks(cl)
   # print("Creating ", cl ," decks")
    #collected_decks = concatenate_deck_class(decks, deck_classes)
    #collected_decks_vectored = concatenate_deck_class(vectorize_deck_list(decks), deck_classes)
    #print("Creating ", cl, ".csv")
    #pd.DataFrame(collected_decks, columns=['cards', 'class']).to_csv("collected_decks_"+deck_type+".csv",index=False)
    #pd.DataFrame(collected_decks_vectored, columns=['cards', 'class']).to_csv("collected_decks_vectored_"+cl+".csv",index=False)



In [10]:
#deck_type = ['Demon Hunter','Druid','Hunter','Mage','Paladin','Priest','Rogue','Shaman','Warlock','Warrior']
deck_type = ['Demon Hunter', 'Druid']
file = open("data.json", "r")
cards = json.load(file)


to_convert = []
for cl in deck_type:
    print("Scraping information: ", cl)
    decks, deck_classes = scrape_decks(cl)
    print("Creating ", cl ," decks")
    for deck in decks:
        for card in deck:
            card = next((new_card for new_card in cards if new_card['name'] == card), None)
    print("Creating ", cl, ".csv")
    to_convert.append(decks)
    #pd.DataFrame(decks, columns=['cards']).to_csv("collected_decks_"+cl+".csv",index=False)


#collected_decks_vectored = concatenate_deck_class(vectorize_deck_list(decks))
#pd.DataFrame(collected_decks, columns=['cards', 'class']).to_csv("collected_decks_"+deck_type+".csv",index=False)
#pd.DataFrame(collected_decks_vectored, columns=['cards', 'class']).to_csv("collected_decks_vectored_"+deck_type+".csv",index=False)



Scraping information:  Demon Hunter
Number of Pages:  58
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=1&filter-class=16384
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=2&filter-class=16384
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=3&filter-class=16384
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=4&filter-class=16384
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=5&filter-class=16384
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=6&filter-class=16384
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=7&filter-class=16384
https://www.hearthpwn.com/decks?f

https://www.hearthpwn.com/decks/1377460-demon-demon-hunter
https://www.hearthpwn.com/decks/1377422-miracle-cthun-dh-mdf
https://www.hearthpwn.com/decks/1377421-garbage-cthun-dh
https://www.hearthpwn.com/decks/1377419-token-demon-hunter
https://www.hearthpwn.com/decks/1377412-highlander-dh
https://www.hearthpwn.com/decks/1377390-midrange-demon-hunter
https://www.hearthpwn.com/decks/1377353-hihglander-demon-hunter
https://www.hearthpwn.com/decks/1377270-highlander-aggro-demon-hunter
https://www.hearthpwn.com/decks/1377346-nmb-mn
https://www.hearthpwn.com/decks/1377345-token-demon-hunter
https://www.hearthpwn.com/decks/1377309-highlander-corrupt-illidari-dh
https://www.hearthpwn.com/decks/1377137-legend-soul-demon-hunter
https://www.hearthpwn.com/decks/1377231-altruis-kek
https://www.hearthpwn.com/decks/1377230-otk
https://www.hearthpwn.com/decks/1376845-giffindex-demon-cthun-75-win-rate-jan-2021
https://www.hearthpwn.com/decks/1376683-pascoas-top-10-otk-dh
https://www.hearthpwn.com/decks

https://www.hearthpwn.com/decks/1372505-four-gods-demon-hunter
https://www.hearthpwn.com/decks/1371997-highlander-dh-is-back
https://www.hearthpwn.com/decks/1372457-big-dh-mdf
https://www.hearthpwn.com/decks/1372454-sold-my-soul-to-reach-legend-for-the-first-time
https://www.hearthpwn.com/decks/1372448-budget-legend-aggro
https://www.hearthpwn.com/decks/1372414-yea
https://www.hearthpwn.com/decks/1372403-trinkiku29s-top-20-tempo-demon-hunter
https://www.hearthpwn.com/decks/1372348-otk-demon-hunter-mdf
https://www.hearthpwn.com/decks/1372333-tempo-demon-hunter
https://www.hearthpwn.com/decks/1372315-budget-rush-deck
https://www.hearthpwn.com/decks/1372292-aggro-outcast-demon-hunter-65-winrate-legend
https://www.hearthpwn.com/decks/1372133-sould-dh-90-winrate-the-best-budget-deck-ever
https://www.hearthpwn.com/decks/1371686-soul-demon-hunter-ilgynoth
https://www.hearthpwn.com/decks/1371607-flowarshsfrs-29-demon-hunter
https://www.hearthpwn.com/decks/1371598-outcast-dh-is-more-than-a-meme

https://www.hearthpwn.com/decks/1365840-ho
https://www.hearthpwn.com/decks/1365855-tempo-dh
https://www.hearthpwn.com/decks/1365749-76-wr-legend-from-diamond-5-soul-demon-hunter
https://www.hearthpwn.com/decks/1365834-death-benefit
https://www.hearthpwn.com/decks/1365807-budget-aggro-dh
https://www.hearthpwn.com/decks/1365593-death-zoo-demon-hunter
https://www.hearthpwn.com/decks/1365607-you-are-not-prepared
https://www.hearthpwn.com/decks/1365675-casual-soul-shard-dh
https://www.hearthpwn.com/decks/1365674-soul-demon-hunter
https://www.hearthpwn.com/decks/1365547-old-guardians-budget-aggro-demon-hunter
https://www.hearthpwn.com/decks/1365240-budget-aggro-demon-hunter
https://www.hearthpwn.com/decks/1365440-tempo-demon-hunter
https://www.hearthpwn.com/decks/1365336-big-demon-demon-hunter
https://www.hearthpwn.com/decks/1365160-demon-hunter-2
https://www.hearthpwn.com/decks/1365159-demon-hunter
https://www.hearthpwn.com/decks/1365107-dh
https://www.hearthpwn.com/decks/1364964-give-me-da

https://www.hearthpwn.com/decks/1360435-soul-hunter
https://www.hearthpwn.com/decks/1360408-soul-shard-dh
https://www.hearthpwn.com/decks/1360393-dh
https://www.hearthpwn.com/decks/1360361-albatross
https://www.hearthpwn.com/decks/1360358-highlander-demon
https://www.hearthpwn.com/decks/1358997-tempo-dh-by-kingozavi
https://www.hearthpwn.com/decks/1358979-sa-aggro-dh
https://www.hearthpwn.com/decks/1360178-aggro-soul-fragment-dh
https://www.hearthpwn.com/decks/1360173-the-best-deck-series-demon-hunter
https://www.hearthpwn.com/decks/1360136-assault-from-the-pit
https://www.hearthpwn.com/decks/1360093-day1-sa-tempo-demon-hunter
https://www.hearthpwn.com/decks/1360088-sertrt
https://www.hearthpwn.com/decks/1360040-21
https://www.hearthpwn.com/decks/1358738-soul-fragment-dh
https://www.hearthpwn.com/decks/1359956-antiaggro
https://www.hearthpwn.com/decks/1359820-soul-fragment
https://www.hearthpwn.com/decks/1359818-sa-cheap-tempo-fragments
https://www.hearthpwn.com/decks/1359774-control-s

...

https://www.hearthpwn.com/decks/1340691-maly-druid
https://www.hearthpwn.com/decks/1340684-token-druid
https://www.hearthpwn.com/decks/1340665-quest-malygos
https://www.hearthpwn.com/decks/1340644-draw-ramp-ysera
https://www.hearthpwn.com/decks/1340610-warning-only-play-if-you-already-have-all-cards
https://www.hearthpwn.com/decks/1340594-choose-the-win
https://www.hearthpwn.com/decks/1340520-highlander-druid
https://www.hearthpwn.com/decks/1340506-trean-spell-druid
https://www.hearthpwn.com/decks/1340443-highlander
https://www.hearthpwn.com/decks/1340422-outland-embiggen-druid
https://www.hearthpwn.com/decks/1340409-big
https://www.hearthpwn.com/decks/1340401-aoo-malygos-druid
https://www.hearthpwn.com/decks/1340369-spell-only-token-druid
https://www.hearthpwn.com/decks/1340366-highlander-token-druid
https://www.hearthpwn.com/decks/1340357-big-dragon-alpha
https://www.hearthpwn.com/decks/1340275-outlands-dragon-ramp-druid
https://www.hearthpwn.com/decks/1340205-aoo-quest-druid
https:/

https://www.hearthpwn.com/decks/1269221-basic-druid
https://www.hearthpwn.com/decks/1269144-willis-druid
https://www.hearthpwn.com/decks/1269137-druidicious
https://www.hearthpwn.com/decks/1257904-f2p-low-budget-druid-for-rise-of-shadow
https://www.hearthpwn.com/decks/1244106-drutros
https://www.hearthpwn.com/decks/1247705-token-druid
https://www.hearthpwn.com/decks/1245616-token
https://www.hearthpwn.com/decks/1243883-druid
https://www.hearthpwn.com/decks/1236031-aoo-quest-druid
https://www.hearthpwn.com/decks/1227954-druida
https://www.hearthpwn.com/decks/1214991-taunt-tastic
https://www.hearthpwn.com/decks/1208239-f2p-taunt-druid
https://www.hearthpwn.com/decks/1179229-druid
https://www.hearthpwn.com/decks/1173882-f2p-beginner-all-basic-druid
https://www.hearthpwn.com/decks/1163954-arbor-druid-featuring-goru-cenarius
https://www.hearthpwn.com/decks/1136327-mana-efficiency-trump
https://www.hearthpwn.com/decks/1137384-lb-druid-missimg
https://www.hearthpwn.com/decks/1130742-beginner-

https://www.hearthpwn.com/decks/99867-basic-druid-for-beginners
https://www.hearthpwn.com/decks/97883-basic-fattie-druid
https://www.hearthpwn.com/decks/91677-basic-druid
https://www.hearthpwn.com/decks/90963-budget-control-druid
https://www.hearthpwn.com/decks/90962-budget-control-druid
https://www.hearthpwn.com/decks/90442-turn-1-taz-dingo
https://www.hearthpwn.com/decks/87438-the-angry-chicken
https://www.hearthpwn.com/decks/86702-combo-wombo
https://www.hearthpwn.com/decks/83142-anubrekkin
https://www.hearthpwn.com/decks/82302-paskos-token-druid
https://www.hearthpwn.com/decks/82101-druid-1
https://www.hearthpwn.com/decks/78764-big-bash-theory-legend
https://www.hearthpwn.com/decks/77420-hearthpwn-deathwing-achievement
https://www.hearthpwn.com/decks/78732-druid-heroic-maexxna-charge-battlecry-mitigation
https://www.hearthpwn.com/decks/78322-naxx-anubrekhan-heroic-fr
https://www.hearthpwn.com/decks/78172-0-dust-to-go-through-all-wing-1-naxx-except
https://www.hearthpwn.com/decks/78

https://www.hearthpwn.com/decks/44454-rank-12-deck-op
https://www.hearthpwn.com/decks/44391-counter-aggro-heal-druid
https://www.hearthpwn.com/decks/43846-greenpeaceoff
https://www.hearthpwn.com/decks/43486-low-budget-druid
https://www.hearthpwn.com/decks/43485-low-budget-druid
https://www.hearthpwn.com/decks/42882-spell-fury
https://www.hearthpwn.com/decks/42863-funny-this-deck-doesnt-look-druish
https://www.hearthpwn.com/decks/42130-minion-burden
https://www.hearthpwn.com/decks/42120-midrange-low-cost
https://www.hearthpwn.com/decks/41538-budget-druid-control
https://www.hearthpwn.com/decks/40939-hobo-legend-killer-aggro-control-druid
https://www.hearthpwn.com/decks/41318-enterthepandas
https://www.hearthpwn.com/decks/41307-low-budget-effective-control-deck
https://www.hearthpwn.com/decks/40570-bloodscrys-rank-5-token-druid-deck
https://www.hearthpwn.com/decks/41102-watcher-druid
https://www.hearthpwn.com/decks/40982-watcher-druid-quills
https://www.hearthpwn.com/decks/40774-kolentod

https://www.hearthpwn.com/decks/1014891-hmm
https://www.hearthpwn.com/decks/1014248-trumps-basic-druid-10-lvl
https://www.hearthpwn.com/decks/986761-new-druid
https://www.hearthpwn.com/decks/978235-htyjhtgrsfe
https://www.hearthpwn.com/decks/963649-druid
https://www.hearthpwn.com/decks/953888-basic-druid
https://www.hearthpwn.com/decks/948847-druid-sdozzo
https://www.hearthpwn.com/decks/931000-cdm4life-cr
https://www.hearthpwn.com/decks/928281-plague-druid
https://www.hearthpwn.com/decks/924211-agro-druida
https://www.hearthpwn.com/decks/917122-seed
https://www.hearthpwn.com/decks/910545-fdas
https://www.hearthpwn.com/decks/899585-ad
https://www.hearthpwn.com/decks/895043-jade-taunt-deathattle-druid
https://www.hearthpwn.com/decks/858531-druida-1
https://www.hearthpwn.com/decks/855398-token-druid
https://www.hearthpwn.com/decks/850666-finja
https://www.hearthpwn.com/decks/848583-custom-druid
https://www.hearthpwn.com/decks/842262-tempo-warrior
https://www.hearthpwn.com/decks/838283-dru

In [36]:

#strip of spaces
def strip(elem):
    elem = elem.rstrip()
    elem = elem.lstrip()
    return elem

#returns json of card name
def get_json(val):
    val = strip(val)
    for card in cards:
        card['name'] = strip(card['name'])
        if card['name'] == val:
            return card
    return '{}'
       

class_json = []
for class_ in to_convert:
    final_json=[]
    for deck in decks:
        deck_list=[]
        for card in deck:
            #print(card)
            json_card = get_json(card)
            #print(list(card.keys())[list(card.values()).index(card))
            #json_card = next((new_card for new_card in cards if new_card['name'] == card), None)
            #print(json_card['name'])
            deck_list.append(json_card)
        final_json.append(deck_list)
    class_json.append(final_json)

In [21]:
#explore the possibility of just doing json.dump like in Card-API
i=0
for i in range(len(class_json)):  
    to_dump = class_json[i];
    print(to_dump[0],"\n")
    with open('data_'+deck_type[i]+'.json', 'w', encoding='utf-8') as f:
        json.dump(to_dump, f, ensure_ascii=False, indent=4)   

[{'cardId': 'DAL_354', 'dbfId': '51790', 'name': 'Acornbearer', 'cardSet': 'Rise of Shadows', 'type': 'Minion', 'rarity': 'Common', 'cost': 1, 'attack': 2, 'health': 1, 'text': '<b>Deathrattle:</b> Add two 1/1 Squirrels to your hand.', 'flavor': 'You’d be nuts to mess with the squirrel squad.', 'artist': 'Steven Prescott', 'collectible': True, 'playerClass': 'Druid', 'img': 'https://d15f34w2p8l1cc.cloudfront.net/hearthstone/e4ed70b1930be57dc0e81ea17f6b83d4aaebd0e877376963f82bfea96017d2d3.png', 'imgGold': 'https://d15f34w2p8l1cc.cloudfront.net/hearthstone/c3548d9c55fc3edb6a5f37e2ed0565172786e2731b8594e9273c6b8158905eb0.png', 'locale': 'enUS', 'mechanics': [{'name': 'Deathrattle'}], 'id': 773}, {'cardId': 'DAL_354', 'dbfId': '51790', 'name': 'Acornbearer', 'cardSet': 'Rise of Shadows', 'type': 'Minion', 'rarity': 'Common', 'cost': 1, 'attack': 2, 'health': 1, 'text': '<b>Deathrattle:</b> Add two 1/1 Squirrels to your hand.', 'flavor': 'You’d be nuts to mess with the squirrel squad.', 'ar

[{'cardId': 'DAL_354', 'dbfId': '51790', 'name': 'Acornbearer', 'cardSet': 'Rise of Shadows', 'type': 'Minion', 'rarity': 'Common', 'cost': 1, 'attack': 2, 'health': 1, 'text': '<b>Deathrattle:</b> Add two 1/1 Squirrels to your hand.', 'flavor': 'You’d be nuts to mess with the squirrel squad.', 'artist': 'Steven Prescott', 'collectible': True, 'playerClass': 'Druid', 'img': 'https://d15f34w2p8l1cc.cloudfront.net/hearthstone/e4ed70b1930be57dc0e81ea17f6b83d4aaebd0e877376963f82bfea96017d2d3.png', 'imgGold': 'https://d15f34w2p8l1cc.cloudfront.net/hearthstone/c3548d9c55fc3edb6a5f37e2ed0565172786e2731b8594e9273c6b8158905eb0.png', 'locale': 'enUS', 'mechanics': [{'name': 'Deathrattle'}], 'id': 773}, {'cardId': 'DAL_354', 'dbfId': '51790', 'name': 'Acornbearer', 'cardSet': 'Rise of Shadows', 'type': 'Minion', 'rarity': 'Common', 'cost': 1, 'attack': 2, 'health': 1, 'text': '<b>Deathrattle:</b> Add two 1/1 Squirrels to your hand.', 'flavor': 'You’d be nuts to mess with the squirrel squad.', 'ar

In [30]:
def json_dump(json_data,name):
    with open('data_'+name+'.json', 'w', encoding='utf-8') as f:
        json.dump(json_data, f, ensure_ascii=False, indent=4)   

In [32]:
i=0
for i in range(len(class_json)):  
    to_dump = class_json[i];
    json_dump(to_dump, deck_type[i])

In [37]:
print(class_json[0][0])


[{'cardId': 'DAL_354', 'dbfId': '51790', 'name': 'Acornbearer', 'cardSet': 'Rise of Shadows', 'type': 'Minion', 'rarity': 'Common', 'cost': 1, 'attack': 2, 'health': 1, 'text': '<b>Deathrattle:</b> Add two 1/1 Squirrels to your hand.', 'flavor': 'You’d be nuts to mess with the squirrel squad.', 'artist': 'Steven Prescott', 'collectible': True, 'playerClass': 'Druid', 'img': 'https://d15f34w2p8l1cc.cloudfront.net/hearthstone/e4ed70b1930be57dc0e81ea17f6b83d4aaebd0e877376963f82bfea96017d2d3.png', 'imgGold': 'https://d15f34w2p8l1cc.cloudfront.net/hearthstone/c3548d9c55fc3edb6a5f37e2ed0565172786e2731b8594e9273c6b8158905eb0.png', 'locale': 'enUS', 'mechanics': [{'name': 'Deathrattle'}], 'id': 773}, {'cardId': 'DAL_354', 'dbfId': '51790', 'name': 'Acornbearer', 'cardSet': 'Rise of Shadows', 'type': 'Minion', 'rarity': 'Common', 'cost': 1, 'attack': 2, 'health': 1, 'text': '<b>Deathrattle:</b> Add two 1/1 Squirrels to your hand.', 'flavor': 'You’d be nuts to mess with the squirrel squad.', 'ar

In [47]:
print(len(class_json[0]))

1696


In [46]:
len(to_convert[0])



1446